In [1]:
import nltk

# Project-Predict job posting is fake or not

aim-In this project we are going to predict job posting is real or fake..
we are using nltk libray for text preprocessing 

loading data

In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
df=pd.read_csv('fake_job_postings.csv')

In [72]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


## Data Processing and Cleaning

#### deal with missing values

In [73]:
df.isna().sum()/len(df)*100

job_id                  0.000000
title                   0.000000
location                1.935123
department             64.580537
salary_range           83.959732
company_profile        18.501119
description             0.005593
requirements           15.072707
benefits               40.324385
telecommuting           0.000000
has_company_logo        0.000000
has_questions           0.000000
employment_type        19.412752
required_experience    39.429530
required_education     45.329978
industry               27.421700
function               36.101790
fraudulent              0.000000
dtype: float64

In [74]:
to_drop = ['job_id','title','department','salary_range','company_profile','requirements','benefits']

df = df.drop(to_drop, axis = 1).sort_index() #drop columns that aren't needed

df = df.dropna(subset = ['description', 'location'])
df.shape

(17533, 11)

In [75]:
df=df[['description','fraudulent']]

In [76]:
df['fraudulent'].value_counts()

0    16687
1      846
Name: fraudulent, dtype: int64

In [7]:
len(df['description'][0])

905

##  Text Processing

1.Removing Punctuations

2.Removing stopwords 

3.Vectorization

4.BOW

5.TF-IDF


In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, naive_bayes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import  nltk

In [10]:
stopwords.words('english')[0:10] 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [77]:
text=df['description'][0]
print(len(text))
text

905


'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff'

In [33]:
stemmer = PorterStemmer() 
stop_words = stopwords.words('english')

def preprocess(text):
    text = re.sub('[^a-zA-Z\s]', '', text) #tokenizatoin
    text = text.lower() #to lower case
    split = text.split() #getting rid of stop words and Porter2 stemming
    for word in split :
      if word in stop_words :
        word = ''
      else :
        stemmer.stem(word)
    return ' '.join([word for word in split])

df['description'] = df['description'].apply(preprocess)

df['description'].head()


0    food a fastgrowing james beard awardwinning on...
1    organised focused vibrant awesomedo you have a...
2    our client located in houston is actively seek...
3    the company esri environmental systems researc...
4    job title itemization review managerlocation f...
Name: description, dtype: object

Creating a function which can removes punctuations and stopwords

In [78]:
def text_preprocess(text):
    new_word=[]
    splited=text.split()
    for i in splited:
        if i not in stopwords.words('english'):
            new_word.append(i)
    return ' '.join(new_word)
    

In [79]:
df['description']=df['description'].apply(text_preprocess)


In [82]:
df['description'][0]

'Food52, fast-growing, James Beard Award-winning online food community crowd-sourced curated recipe hub, currently interviewing full- part-time unpaid interns work small team editors, executives, developers New York City headquarters.Reproducing and/or repackaging existing Food52 content number partner sites, Huffington Post, Yahoo, Buzzfeed, various content management systemsResearching blogs websites Provisions Food52 Affiliate ProgramAssisting day-to-day affiliate program support, screening affiliates assisting affiliate inquiriesSupporting PR &amp; Events neededHelping office administrative work, filing, mailing, preparing meetingsWorking developers document bugs suggest improvements siteSupporting marketing executive staff'

In [83]:
len(df['description'].head()[0])

736

In [84]:
len(df['description'][0])

736

### Vectorization 

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

In [86]:
df.isnull().sum()/len(df) *100

description    0.0
fraudulent     0.0
dtype: float64

In [103]:
df['description']


0        food fastgrowing james beard awardwinning onli...
1        organised  focused  vibrant  awesomedo passion...
2        our client located houston actively seeking ex...
3        the company esri  environmental systems resear...
4        job title itemization review managerlocation f...
                               ...                        
17875    just case first time youve visited website ven...
17876    the payroll accountant focus primarily payroll...
17877    experienced project cost control staff enginee...
17878    nemsia studios looking experienced visualgraph...
17879    who wevend award winning web based point sale ...
Name: description, Length: 17533, dtype: object

In [104]:
bow_transformer = CountVectorizer().fit(df['description'])
print(len(bow_transformer.vocabulary_))

93785


In [105]:
text_bow= bow_transformer.transform(df['description'])

In [106]:
print(text_bow.shape)

(17533, 93785)


### Tf-Idf

In [107]:
from sklearn.feature_extraction.text import TfidfTransformer

In [108]:
tfidf_transformer= TfidfTransformer().fit(text_bow)

In [109]:
stemmer = PorterStemmer() 
stop_words = stopwords.words('english')

def preprocess(text):
    text = re.sub('[^a-zA-Z\s]', '', text) #tokenizatoin
    text = text.lower() #to lower case
    
    return text
df['description'] = df['description'].apply(preprocess)

df['description'].head()


0    food fastgrowing james beard awardwinning onli...
1    organised  focused  vibrant  awesomedo passion...
2    our client located houston actively seeking ex...
3    the company esri  environmental systems resear...
4    job title itemization review managerlocation f...
Name: description, dtype: object

## Preparation of Dataset

In [111]:
#define the variables
x=df['description']
y=df['fraudulent']

#split data into training and test set


x_train, x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.2,random_state=42)

#encoding
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

#vectorization
MAX=2000
vectorizer=TfidfVectorizer(max_features=MAX)
vectorizer.fit(x_train)

x_trainvec=vectorizer.transform(x_train)
x_testvec=vectorizer.transform(x_test)



In [151]:
x_trainvec.shape

(14026, 2000)

In [132]:
y.value_counts()

0    16687
1      846
Name: fraudulent, dtype: int64

# Model

#### Logistic Regression

In [112]:
from sklearn.linear_model import LogisticRegression

In [113]:
#fitting
logreg=LogisticRegression()
logreg.fit(x_trainvec,y_train)

#predict
y_pred_lr=logreg.predict(x_testvec)



In [114]:
print("Accurancy score of logistic regression",accuracy_score(y_pred_lr,y_test))
print("Confusion matrix of logistic regression",confusion_matrix(y_pred_lr,y_test))

Accurancy score of logistic regression 0.9626461362988309
Confusion matrix of logistic regression [[3322  129]
 [   2   54]]


In [115]:
print("classification report of logistic regression:",classification_report(y_pred_lr,y_test))

classification report of logistic regression:               precision    recall  f1-score   support

           0       1.00      0.96      0.98      3451
           1       0.30      0.96      0.45        56

    accuracy                           0.96      3507
   macro avg       0.65      0.96      0.72      3507
weighted avg       0.99      0.96      0.97      3507



# KNN 

In [116]:
from sklearn.neighbors import KNeighborsClassifier


In [117]:
knn=KNeighborsClassifier(n_neighbors=5)

In [118]:
knn.fit(x_trainvec,y_train)

KNeighborsClassifier()

In [119]:
y_pred_knn=knn.predict(x_testvec)

In [120]:
print("Accuray score of KNN",accuracy_score(y_pred_knn,y_test))

Accuray score of KNN 0.9726261762189906


#### choosing k value

In [121]:
#error=[]
#or i in range(n_)

## SVC

In [122]:
from sklearn.svm import SVC

In [123]:
svc=SVC(kernel='rbf')
svc.fit(x_trainvec,y_train)
y_pred_svc=svc.predict(x_testvec)

In [124]:
accuracy_score(y_pred_svc,y_test)

0.9726261762189906

In [125]:
print(classification_report(y_pred_svc,y_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99      3420
           1       0.48      1.00      0.64        87

    accuracy                           0.97      3507
   macro avg       0.74      0.99      0.82      3507
weighted avg       0.99      0.97      0.98      3507

